In [93]:
import pandas as pd
import numpy as np
import pickle
import math
from StringContains import ContainedStringCase 
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt
plt.style.use('seaborn-paper')
import os
from scipy.stats import ks_2samp, ttest_rel, f, mannwhitneyu, pearsonr, spearmanr, kendalltau
from jmh_parser import parseFile
from benchmarkutils import get_data_frame_from_csv, get_ks_2sample_test_log, get_mann_whitney_u_test_test_log, read_log
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [150]:
d = pickle.load(open("http_nonmatching_strings.input9", "rb"))
for gen_str, str_len, match_pos_ratio in d:
    print(repr(gen_str), str_len)

't[#W' 4
'ls~,' 4
'Oh3D' 4
'@0zS' 4
'2EbZvL@\t' 8
'Kf?^\x0bs5Y' 8
'Mp73_,(f' 8
'-(0i#.U6' 8
'gfg2MN~pg3;:Q`y[' 16
'J3e!3];o\r+pPdgUJ' 16
'1PCcUBo\nt?;cVs+f' 16
'.TB\rT7(S^+uc\\@=X' 16
'j]g,#/U]<7Dx0wE| s\\JC%nQ,~\x0bMt9eZ' 32
'As N3K#qLy)4)xT;)\x0cx;yM<Sr\rK2oFl<' 32
'lZXy+L,2& KdeJ3q%?$KooEkC5DB<Tqe' 32
'h@pGVj:v/SI>I9Pa{ja\t*xTn0G"w+)z{' 32
'}XWYw\rsYq1n,,qZMFEI&zD|pk>\nne@f\t@z\x0bDqK6U(W7F"a0]T{>{sT?l\x0b95O"|wq' 64
'iCI*Dc@6BgoX2}8^7P@;Z>!r8.dNmfc;)3%Vt|\r#-&Xr$Fj7Le"{Uve|[k5l~]\rC' 64
"'nI5{,@*Px2c([\x0b7 Pt/E,D'5U\tzk\tFv$A[{Fmvgp03D:pO yg@mBR/(>l1a*(@|" 64
'#mL\no*|\x0b&^T@1$6l<?U$sTd6j(N^l$c"JHUV-UG"~x;+g<\x0bmQ5Dy=K#\r>\'_\t<\x0c^B' 64
'lJ^JiHCqD,<?t^qE"dGQ)((|/Ns^dxHJN_\x0b;"fJw[[s1P]hj\'Fjg1[YZf*U3ul"j*b|:O\n=S#RMQZ\x0cfr>-DRRhn(Sj1j0\nRt.&WlvxW-+,83;=~BG4(g\r-KT(|K/bV#z' 128
"K;P\x0ctTn%[GmJ7Q*tD>4T+1\tR&7+2AbqO>S8(h\x0bwDpRJ\x0cdbvrgW[Zn6]$|)2<`[Go>'`~vot>13+,JX2 D/Y`4|@[s{cnYMO0s(|E]:Q50%\r':UlCk9sG9T>G5qOt!Rrb" 128
'~76T8P&41,FQW:}FEN<!ZJFYag6)9J"`hj%\nUew!Skq\x0cAC{\nC

In [94]:
dir1 = "same_len_warmup3_measurement10_time1/"
dir2 = "same_len_warmup5_measurement20_time1/"
dir3 = "same_len_warmup5_measurement20_time1_dataset2/"
#dir3 = 
data = pickle.load(open("http_test_10runs.input2", "rb"))

In [95]:
t = pd.read_csv(dir1+"948_True_1_run.csv")
t.columns

Index(['Benchmark', 'Mode', 'Threads', 'Samples', 'Score',
       'Score Error (99.9%)', 'Unit', 'Param: regex', 'Param: str'],
      dtype='object')

In [143]:
def read_csv(dir_path, str_len, matches, true_len, file_prefix = None):
    data = []
    for idx in range(10):
        file_name = '_'.join([str(str_len), str(matches), str(idx), "run"]) + ".csv"
        if file_prefix:
            file_name = file_prefix + file_name
        if not os.path.exists(dir_path + file_name):
            print(file_name, "is missing")
            continue
        df = pd.read_csv(dir_path + file_name)
        df = df.loc[:, ["Benchmark", "Score", 'Score Error (99.9%)', "Unit"]]
        df['Benchmark'] = df["Benchmark"].map(lambda x: x.split(".")[-1])
        df["matches"] = matches
        df['str_len'] = true_len
        df['run'] = idx
#         df['Score'] = df['Score'].round().astype(int)
        data.append(df)
    df = pd.concat(data, ignore_index=True)
    df = df.pivot_table(values = "Score", index=["str_len","matches","run"], columns="Benchmark")
    df.reset_index(drop=False, inplace=True) 
    return df

In [97]:
for gen_str, str_len, matches, true_len in data:
    print(str_len, matches, true_len)
    df1 = read_csv(dir1, str_len, matches, true_len)
    df2 = read_csv(dir2, str_len, matches, true_len)
    for method in ["regexMatches", "stringContains"]:
#         print("http", method, ks_2samp(df1[method],df2[method]))
#         print(method, "standard deviation", df1[method].std(), df2[method].std())
#         print(method, "mean", df1[method].mean(), df2[method].mean())
        print(method, "relative deviation", df1[method].std() / df1[method].mean(), df2[method].std() / df2[method].mean())
        

9 True 15
regexMatches relative deviation 0.09148974347811364 0.07020918394760046
stringContains relative deviation 0.06200544431702704 0.0
9 False 15
regexMatches relative deviation 0.016899279413057473 0.03413646149347612
stringContains relative deviation 0.0 0.0
17 True 10
regexMatches relative deviation 0.06028868856739036 0.04662523600892256
stringContains relative deviation 0.0 0.0
17 False 10
regexMatches relative deviation 0.0214701816761287 0.022704073355787592
stringContains relative deviation 0.0 0.0
63 True 23
regexMatches relative deviation 0.09614889120418758 0.04632553430974832
stringContains relative deviation 0.0 0.0
63 False 23
regexMatches relative deviation 0.01968805310939817 0.022473094460020396
stringContains relative deviation 0.0 0.0
125 True 52
regexMatches relative deviation 0.0888237451005817 0.0507314054146132
stringContains relative deviation 0.03377943297480055 0.03586095690932794
125 False 52
regexMatches relative deviation 0.07437861379670956 0.07447900

In [98]:
data1 = pickle.load(open("http_test_10runs.input4", "rb"))
data2 = pickle.load(open("http_test_10runs.input5", "rb"))

In [145]:
for d1, d2 in zip(data1, data2):
    gen_str, str_len, matches, true_len = d1
    if (matches, true_len) in [(False, 90)]:
        print(len(d1[0]), len(d2[0]))
        print(repr(d1[0]))
        print(repr(d2[0]))

90 90
':PiTY28Wd$s5Xz1&o*rk}9|(np\x0c]t+{@3PsLuM.SuC_n\x0bi&ZQ^-rVwqRA-\tD)t\tN`6R0jg-xx^\t8\tBb&CQbdw|;WR7'
'A.Bn^jOY6\x0bZuvGz+wP(\'\x0cj`+Y$ n27Fg9v \n(hcVCvN`SHMHhkCgL!\nqpf{|cYsx"(@dwe}qm>\ri=**#1aX4I_wOJU'


In [106]:
for d1, d2 in zip(data1, data2):
    gen_str, str_len, matches, true_len = d1
    print(str_len, matches, true_len)
    if not matches:
        print(len(d1[0]) == len(d2[0]))
    else:
        match_pos = gen_str.index("http")
        print(len(d1[0]) == len(d2[0]), match_pos == d2[0].index("http"))

9 True 9
True True
9 False 9
True
17 True 18
True True
17 False 18
True
63 True 90
True True
63 False 90
True
125 True 137
True True
125 False 137
True
318 True 318
True True
318 False 318
True
948 True 1537
True True
948 False 1537
True


In [120]:
dir1 = "dataset4/"
dir2 = "dataset5/"
count_t, count_ks = 0, 0
for gen_str, str_len, matches, true_len in data1:
    print(str_len, matches, true_len)
    df1 = read_csv(dir1, str_len, matches, true_len, "dataset4_")
    df2 = read_csv(dir2, str_len, matches, true_len, "dataset5_")
    for method in ["regexMatches", "stringContains"]:
        t = ks_2samp(df1[method],df2[method])
        t_test_result = ttest_rel(df1[method],df2[method])
#         stat, p = mannwhitneyu(df1[method],df2[method])
        print("http", method, t[1], t_test_result[1], t[1] < 0.05, t_test_result[1] < 0.05)
#         print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
#         print("http", method, t, t[1] < 0.05)
#         if t[1] < 0.05:
#             print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#             count_ks += 1

#         print("http", method, t_test_result, t_test_result[1] < 0.05)
#         if not np.isnan(t_test_result[1]) and t_test_result[1] < 0.05:
#             print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#             count_t += 1
# print(count_ks)
        

9 True 9
http regexMatches 0.16782134274394334 0.1764683890244105 False False
http stringContains 1.0 nan False False
9 False 9
http regexMatches 0.9944575548290717 0.34343639613791355 False False
http stringContains 0.9944575548290717 0.17726770250560825 False False
17 True 18
http regexMatches 0.41752365281777043 0.1559460786300557 False False
http stringContains 1.0 nan False False
17 False 18
http regexMatches 0.7869297884777761 0.3213819886045159 False False
http stringContains 0.9944575548290717 0.1934220596033321 False False
63 True 90
http regexMatches 0.16782134274394334 0.41674064682571355 False False
http stringContains 1.0 0.34343639613791355 False False
63 False 90
http regexMatches 1.0825088224469026e-05 7.401297753730516e-15 True True
http stringContains 1.0 0.34343639613791355 False False
125 True 137
http regexMatches 0.41752365281777043 0.4419014607691555 False False
http stringContains 1.0825088224469026e-05 1.458947122944854e-06 True True
125 False 137
http regexMat

In [92]:
for d1, d2 in zip(data1, data2):
    gen_str, str_len, matches, true_len = d1
#     print(str_len, matches, true_len)
    if (matches, true_len) in [(False, 90)]: #, (True, 137), (False, 137), (False, 318), (True, 1537), (False, 1537)]:
        print(len(d1[0]), len(d2[0]))

90 90


In [136]:
df1 = read_csv("sample1/", 63, False, 90, "dataset4_")
df2 = read_csv("sample1/", 63, False, 90, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
#     print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
#     print("http", method, t[1], t_test_result[1], t[1] < 0.05, t_test_result[1] < 0.05)
#     print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
    stat, p = mannwhitneyu(df1[method],df2[method])
    print(method, stat, p, not np.isnan(p) and p < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#     print("http", method, t, t[1] < 0.05)
#     if t[1] < 0.05:
# #             print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         count_ks += 1

#         print("http", method, t_test_result, t_test_result[1] < 0.05)
#         if not np.isnan(t_test_result[1]) and t_test_result[1] < 0.05:
# #             print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#             count_t += 1

regexMatches 0.0 7.966008837031103e-05 True
   data1  data2
0    230    601
1    230    602
2    231    604
3    232    601
4    232    602
5    208    601
6    207    604
7    230    602
8    213    602
9    208    604


ValueError: All numbers are identical in mannwhitneyu

In [144]:
df1 = read_csv("samples3/", 125, True, 137, "dataset4_")
df2 = read_csv("samples3/", 125, True, 137, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print(method, stat, p, not np.isnan(p) and p < 0.05)
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))

regexMatches 0.0 7.966008837031103e-05 True
http regexMatches KstestResult(statistic=0.3, pvalue=0.7869297884777761) Ttest_relResult(statistic=-0.3865724087759458, pvalue=0.7080537004592232) False False
        data1       data2
0  446.725351  482.624198
1  447.082527  387.065781
2  388.167144  444.746230
3  472.127657  447.187311
4  448.456914  446.229891
5  434.289361  366.991991
6  387.900333  456.892414
7  447.391057  483.121310
8  446.552405  446.634063
9  459.713799  473.303946
stringContains 0.0 7.966008837031103e-05 True
http stringContains KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-23.454061418807495, pvalue=2.218567071928977e-09) True True
       data1      data2
0  23.191681  24.600635
1  23.202841  24.582743
2  23.185349  24.889049
3  23.207284  24.581978
4  23.206262  24.596051
5  23.214260  24.564698
6  23.209238  24.585821
7  23.235300  24.586207
8  23.222859  24.591135
9  23.654463  24.588032


In [137]:
df1 = read_csv("sample2/", 125, True, 137, "dataset4_")
df2 = read_csv("sample2/", 125, True, 137, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print(method, stat, p, not np.isnan(p) and p < 0.05)
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         stat, p = mannwhitneyu(df1[method],df2[method])
    

dataset4_125_True_6_run.csv is missing
dataset5_125_True_4_run.csv is missing
regexMatches 0.0 7.966008837031103e-05 True
http regexMatches KstestResult(statistic=0.3333333333333333, pvalue=0.7301110654051831) Ttest_relResult(statistic=-0.24653848544132304, pvalue=0.8114736878866722) False False
   data1  data2
0    448    446
1    436    445
2    388    367
3    466    471
4    450    459
5    436    367
6    448    457
7    346    457
8    473    457
stringContains 0.0 7.966008837031103e-05 True
http stringContains KstestResult(statistic=1.0, pvalue=4.11353352529823e-05) Ttest_relResult(statistic=-17.0, pvalue=1.4552108482941523e-07) True True
   data1  data2
0     23     25
1     23     25
2     23     25
3     23     25
4     23     25
5     23     25
6     23     25
7     23     25
8     24     25


In [126]:
df1 = read_csv("sample3/", 125, False, 137, "dataset4_")
df2 = read_csv("sample3/", 125, False, 137, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         stat, p = mannwhitneyu(df1[method],df2[method])
    

http regexMatches KstestResult(statistic=0.9, pvalue=0.00021650176448938054) Ttest_relResult(statistic=-1.9369746332504625, pvalue=0.08471955073794296) True False
   data1  data2
0    867    884
1    866    883
2    865    820
3    866    889
4    866    884
5    866    885
6    867    886
7    866    884
8    865    883
9    866    886
http stringContains KstestResult(statistic=0.2, pvalue=0.9944575548290717) Ttest_relResult(statistic=-1.4999999999999998, pvalue=0.16785065605707492) False False
   data1  data2
0     27     27
1     27     27
2     27     28
3     27     27
4     27     27
5     27     28
6     27     27
7     27     27
8     27     27
9     27     27


In [124]:
df1 = read_csv("sample4/", 318, False, 318, "dataset4_")
df2 = read_csv("sample4/", 318, False, 318, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         stat, p = mannwhitneyu(df1[method],df2[method])
    

http regexMatches KstestResult(statistic=0.5, pvalue=0.16782134274394334) Ttest_relResult(statistic=-1.3651682409395072, pvalue=0.2053486331553807) False False
   data1  data2
0   1819   1823
1   1821   1950
2   1810   1821
3   1818   1822
4   1823   1826
5   1828   1824
6   1813   1824
7   1822   1824
8   1812   1822
9   1826   1827
http stringContains KstestResult(statistic=0.1, pvalue=1.0) Ttest_relResult(statistic=1.0, pvalue=0.34343639613791355) False False
   data1  data2
0     53     52
1     52     52
2     52     52
3     52     52
4     52     52
5     52     52
6     52     52
7     52     52
8     52     52
9     52     52


In [122]:
df1 = read_csv("sample5/", 948, True, 1537, "dataset4_")
df2 = read_csv("sample5/", 948, True, 1537, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         stat, p = mannwhitneyu(df1[method],df2[method])

http regexMatches KstestResult(statistic=0.5, pvalue=0.16782134274394334) Ttest_relResult(statistic=-0.6711979587923576, pvalue=0.5189478900856783) False False
   data1  data2
0   7059   6682
1   6738   7771
2   7054   7765
3   7063   7031
4   7053   7027
5   7799   7035
6   6735   7039
7   7056   7030
8   7055   7028
9   6740   7031
http stringContains KstestResult(statistic=0.1, pvalue=1.0) Ttest_relResult(statistic=-1.0, pvalue=0.34343639613791355) False False
   data1  data2
0     90     90
1     90     90
2     91     91
3     90     90
4     91    108
5     90     90
6     90     90
7     90     90
8     90     90
9     90     90


In [121]:
df1 = read_csv("sample6/", 948, False, 1537, "dataset4_")
df2 = read_csv("sample6/", 948, False, 1537, "dataset5_")
for method in ["regexMatches", "stringContains"]:
    t = ks_2samp(df1[method],df2[method])
    t_test_result = ttest_rel(df1[method],df2[method])
    print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
    print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))
#         stat, p = mannwhitneyu(df1[method],df2[method])

http regexMatches KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-2.896947983924918, pvalue=0.017683035369055917) True True
   data1  data2
0   8963   9083
1   8389   9088
2   8970   9052
3   8972   9088
4   8967   9091
5   8383   9051
6   8971   9090
7   8970   9089
8   8967   9058
9   8962   9053
http stringContains KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=29.849623113198597, pvalue=2.597549811426689e-10) True True
   data1  data2
0    232    225
1    231    225
2    232    225
3    231    225
4    232    225
5    231    226
6    232    225
7    232    225
8    232    225
9    232    225


In [138]:
for str_len, matches, true_len in [(63, False, 90), (125, True, 137), (125, False, 137), (948, False, 1537)]:
    print(str_len, matches, true_len)
    df1 = read_csv("samples/", str_len, matches, true_len, "dataset4_")
    df2 = read_csv("samples/", str_len, matches, true_len, "dataset5_")
    for method in ["regexMatches", "stringContains"]:
        t = ks_2samp(df1[method],df2[method])
        t_test_result = ttest_rel(df1[method],df2[method])
        if t[1] < 0.05 or (not np.isnan(t_test_result[1]) and t_test_result[1] < 0.05):
            print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
#             print("http", method, t[1], t_test_result[1], t[1] < 0.05, t_test_result[1] < 0.05)
            print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))

63 False 90
http regexMatches KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-72.09530191447662, pvalue=9.608032025724002e-14) True True
   data1  data2
0    171    460
1    175    461
2    144    461
3    150    460
4    176    460
5    150    459
6    150    461
7    151    460
8    171    459
9    172    459
125 True 137
http regexMatches KstestResult(statistic=0.8, pvalue=0.002056766762649115) Ttest_relResult(statistic=-2.7687578028343798, pvalue=0.02179773634331166) True True
   data1  data2
0    326    347
1    339    345
2    263    353
3    336    355
4    346    346
5    337    347
6    337    345
7    326    346
8    260    334
9    336    353
http stringContains KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-11.000000000000002, pvalue=1.609932479393564e-06) True True
   data1  data2
0     18     19
1     18     19
2     18     19
3     18     19
4     17     19
5     18     19
6     18     19
7     

In [141]:
for str_len, matches, true_len in [(63, False, 90), (125, True, 137), (125, False, 137), (948, False, 1537)]:
    print(str_len, matches, true_len)
    df1 = read_csv("samples2/", str_len, matches, true_len, "dataset4_")
    df2 = read_csv("samples2/", str_len, matches, true_len, "dataset5_")
    for method in ["regexMatches", "stringContains"]:
        t = ks_2samp(df1[method],df2[method])
        t_test_result = ttest_rel(df1[method],df2[method])
        if t[1] < 0.05 or (not np.isnan(t_test_result[1]) and t_test_result[1] < 0.05):
            print("http", method, t, t_test_result, t[1] < 0.05, t_test_result[1] < 0.05)
#             print("http", method, t[1], t_test_result[1], t[1] < 0.05, t_test_result[1] < 0.05)
            print(pd.DataFrame({"data1":df1[method], "data2":df2[method]}))

63 False 90
http regexMatches KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-96.99755785867572, pvalue=6.6730582659712116e-15) True True
   data1  data2
0    233    602
1    230    601
2    208    604
3    230    601
4    208    608
5    208    601
6    209    601
7    230    603
8    208    604
9    209    601
125 True 137
http stringContains KstestResult(statistic=1.0, pvalue=1.0825088224469026e-05) Ttest_relResult(statistic=-13.499999999999998, pvalue=2.806720467052002e-07) True True
   data1  data2
0     23     25
1     23     25
2     24     25
3     24     25
4     23     25
5     23     25
6     23     25
7     23     25
8     23     25
9     23     25
125 False 137
http regexMatches KstestResult(statistic=0.9, pvalue=0.00021650176448938054) Ttest_relResult(statistic=-2.03787918152945, pvalue=0.07201158495217266) True False
   data1  data2
0    865    824
1    866    883
2    865    883
3    868    885
4    869    889
5    865    883
6    8